<a href="https://www.kaggle.com/prosperalikizang/tmnist?scriptVersionId=90296821" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# TMNIST (Typography MNIST) Glyphs: A database of over 500,000 MNIST style images made from 1,812 unique glyphs and 3,019 font-styles


In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# EDA

## Libraries and Data Import 

In [ ]:
import numpy as np
import pandas as pd
import os
import random
import cv2
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm


data = pd.read_csv('../input/tmnist-glyphs-1812-characters/Glyphs_TMNIST_updated.csv')
ft_lb=data[['font_name','label']]
typ=ft_lb.groupby('font_name',as_index=False).sum()
typ 

In [ ]:
for i in range(13):     # upto 3019
    print(typ.iloc[i,0])
    print(typ.iloc[i,1])
    print()

In [ ]:
del typ
del ft_lb

## Features engineering

In [ ]:
# Using of tqdm to see a progress bar 
from tqdm import tqdm
drop=[]
for i in tqdm(range(len(data))):
    item=data['label'][i]
    if item not in 'ČĆĐŠŽčćđšž₹ĂÂÊÔƠƯăâêôơưАБВГҐДЂЕЁЄЖЗЅИІЇЙЈКЛЉМНЊОПРСТЋУЎФХЦЧЏШЩЪЫЬЭЮЯабвгґдђеёєжзѕиіїйјклљмнњопрстћуўфхцчџшщъыьэюя':
        drop+=[i]


In [ ]:
f_data=data.drop(index=data.index[drop],axis=0).reset_index(drop=True)
f_data

In [ ]:
del data

In [ ]:
Name=[]
char='ČĆĐŠŽčćđšž₹ĂÂÊÔƠƯăâêôơưАБВГҐДЂЕЁЄЖЗЅИІЇЙЈКЛЉМНЊОПРСТЋУЎФХЦЧЏШЩЪЫЬЭЮЯабвгґдђеёєжзѕиіїйјклљмнњопрстћуўфхцчџшщъыьэюя'
for i in range(len(char)):
    Name+=[char[i]]
print(Name)
print(len(Name))

In [ ]:
M=list(range(len(Name)))
normal_mapping=dict(zip(Name,M)) 
reverse_mapping=dict(zip(M,Name)) 

In [ ]:
n=len(f_data)
Y_train= f_data['label'][0:(n//4)*3].map(normal_mapping).reset_index(drop=True)
Y_test= f_data['label'][(n//4)*3:].map(normal_mapping).reset_index(drop=True)

In [ ]:
l_data= f_data.drop(['glyph_name','font_name','label'],axis=1)
X_train= l_data.loc[0:(n//4)*3-1].reset_index(drop=True)
X_test= l_data.loc[(n//4)*3:].reset_index(drop=True)

In [ ]:
del f_data

In [ ]:
# Encoding the label
binencoder = LabelBinarizer()
y0 = Y_train
y = binencoder.fit_transform(y0)
y

In [ ]:
X_images = X_train.values.reshape(-1,28,28)
test_images = X_test.values.reshape(-1,28,28)
print(X_images.shape)
print(test_images.shape)

In [ ]:
fig, axs = plt.subplots(3,3,figsize=(12,12))
for i in range(9):
    r=i//3
    c=i%3
    ax=axs[r][c].axis("off")
    actual=reverse_mapping[Y_train[i]]     
    ax=axs[r][c].set_title(actual)
    ax=axs[r][c].imshow(X_images[i])
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_images, y, test_size=0.2, random_state=90)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
X_train = (X_train/255).reshape(-1,28,28,1).astype('float32')
X_test = (X_test/255).reshape(-1,28,28,1).astype('float32')

In [ ]:
test_images = (test_images/255).reshape(-1,28,28,1).astype('float32')

## Building the model

## Training with the first model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.callbacks import EarlyStopping
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

In [ ]:
# model = Sequential()
# model.add(Conv2D(32,(4,4),input_shape = (28,28,1),activation = 'relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Conv2D(64,(3,3),activation = 'relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.2))
# model.add(Flatten())
# model.add(Dense(len(Name)*4,activation='relu'))
# model.add(Dense(len(Name)*2, activation='relu'))
# model.add(Dense(len(Name), activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
# model.summary()

In [ ]:
# cnn = model.fit(X_train, y_train, validation_split=0.2, epochs=20, batch_size=92, verbose=2)
# cnn

In [ ]:
# his = pd.DataFrame(cnn.history)
# his.loc[:, ['accuracy', 'val_accuracy']].plot()
# his.loc[:, ['loss', 'val_loss']].plot()

## Training with second model

In [ ]:
from keras.models import Sequential
from keras import optimizers
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense, Reshape
from keras import backend as K
from keras.constraints import maxnorm
from keras.utils import np_utils
# Set the CNN model 
# Our CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out

model = Sequential()

model.add(Convolution2D(32, (5,5), input_shape=(28,28,1),
                             activation='relu',padding='same',
                            kernel_constraint=maxnorm(3)))
model.add(Convolution2D(32, (5,5), input_shape=(28,28,1),
                             activation='relu',
                            kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, (3,3), input_shape=(28,28,1),
                             activation='relu',padding='same',
                            kernel_constraint=maxnorm(3)))
model.add(Convolution2D(64, (3,3), input_shape=(28,28,1),
                             activation='relu',
                            kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(113, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train, y_train,
                        batch_size=128, epochs=20)

In [ ]:
from matplotlib import pyplot as plt
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.grid()
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.grid()
plt.show()

## Predictions

In [ ]:
y_pred = model.predict(test_images)
print(y_pred[0])

In [ ]:
y_pred = model.predict(test_images)
print(y_pred[0])

In [ ]:
y_pred2=[]
for item in y_pred:   
    y_pred2+=[np.argmax(item)   ]
PRED=np.array(y_pred2)
ANS=np.array(Y_test)
print(PRED[0:10])
print(ANS[0:10])

In [ ]:
y_pred2=[]
for item in y_pred:   
    y_pred2+=[np.argmax(item)   ]
PRED=np.array(y_pred2)
ANS=np.array(Y_test)
print(PRED[0:10])
print(ANS[0:10])

In [ ]:
accuracy_score(ANS,PRED)

In [ ]:
accuracy_score(ANS,PRED)

In [ ]:
fig, axs = plt.subplots(3,3,figsize=(12,12))
for i in range(9):
    r=i//3
    c=i%3
    ax=axs[r][c].axis("off")
    actual=reverse_mapping[Y_test[i]]   
    predict=reverse_mapping[PRED[i]]    
    ax=axs[r][c].set_title(actual+'=='+predict)
    ax=axs[r][c].imshow(test_images[i])
plt.show()

## Saving the model

In [ ]:
from keras.models import load_model
from keras.models import model_from_json

#saves the model info as json file
history_model_0_95 = model.to_json()
with open("history_model_0_95.json", "w") as json_file:
    json_file.write(history_model_0_95)
 

# Creates a HDF5 file 'model.h5'
model.save_weights("history_model_0_95.h5")
print("Saved model to disk")

In [ ]:
#evaluating model on test data. will take time
scores = model.evaluate(X_test,y_test, verbose = 0)
print("Accuracy: %.2f%%"%(scores[1]*100))